In [116]:
import os
import re
import json
import glob
import random
from PIL import Image, ImageDraw
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [117]:
from annoy import AnnoyIndex

In [118]:
import torch
from torch.utils import data
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn

In [119]:
import torch
from torch.utils import data
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn

In [120]:
data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [121]:
# loading the trained model and generating embedding based on that
base_model = models.resnet18(pretrained=True)
for param in base_model.parameters():
    param.requires_grad = False
num_ftrs = base_model.fc.in_features
base_model.fc = nn.Sequential(nn.Linear(num_ftrs, 256), nn.Linear(256, 128))

# loading the trained model with trained weights
checkpoint = torch.load("./weights/model_best.pth")
base_model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [122]:
base_model = base_model.cpu()
base_model = base_model.eval()

In [123]:
annoy_index = AnnoyIndex(128, metric="euclidean")
annoy_index.load("./annoy_index.ann")


True

In [124]:
with open('annoy_index_to_label.json') as f:
    annoy_index_to_label = json.load(f)

In [125]:
query_img_name = './dataset/val_retina/4.png'

In [126]:
query_img = Image.open(query_img_name)
query_img = data_transform(query_img)
query_img = query_img.unsqueeze(0)
query_img_embedding = base_model(query_img)
query_img_embedding = query_img_embedding.squeeze()
query_img_embedding.shape

torch.Size([128])

In [127]:
query_img_embedding

tensor([-0.0396, -0.0635,  0.0229,  0.0253,  0.0816, -0.3173, -0.3418, -0.2621,
         0.3287,  0.2832,  0.2842,  0.0544,  0.1584, -0.0022,  0.5344, -0.0097,
         0.2373, -0.1543,  0.4458, -0.0574,  0.0838,  0.3555, -0.1194, -0.0714,
         0.1287, -0.0155,  0.0211,  0.1945,  0.2851, -0.0452,  0.0064,  0.2497,
         0.3413,  0.0381, -0.0924,  0.1646,  0.0217, -0.0858, -0.1268,  0.2020,
        -0.0409, -0.0884,  0.0089,  0.0843, -0.1467,  0.0396,  0.1318, -0.4844,
        -0.0322, -0.4791,  0.2510, -0.3303,  0.4215, -0.0503,  0.2348, -0.3210,
         0.1912, -0.0605, -0.2135,  0.1815,  0.2225, -0.0769,  0.0436,  0.0426,
         0.0520, -0.1147,  0.1500, -0.1225, -0.0742,  0.1197,  0.1429,  0.1736,
         0.1802, -0.2868,  0.1878,  0.2693, -0.0378, -0.0430, -0.1295,  0.0281,
        -0.0408,  0.1477, -0.0142,  0.0383,  0.0363,  0.3557,  0.2661,  0.0890,
        -0.1227, -0.0353, -0.0621, -0.0224,  0.1996, -0.1658,  0.1955, -0.2141,
         0.1688,  0.1162, -0.0088,  0.21

In [128]:
query_img_embedding

tensor([-0.0396, -0.0635,  0.0229,  0.0253,  0.0816, -0.3173, -0.3418, -0.2621,
         0.3287,  0.2832,  0.2842,  0.0544,  0.1584, -0.0022,  0.5344, -0.0097,
         0.2373, -0.1543,  0.4458, -0.0574,  0.0838,  0.3555, -0.1194, -0.0714,
         0.1287, -0.0155,  0.0211,  0.1945,  0.2851, -0.0452,  0.0064,  0.2497,
         0.3413,  0.0381, -0.0924,  0.1646,  0.0217, -0.0858, -0.1268,  0.2020,
        -0.0409, -0.0884,  0.0089,  0.0843, -0.1467,  0.0396,  0.1318, -0.4844,
        -0.0322, -0.4791,  0.2510, -0.3303,  0.4215, -0.0503,  0.2348, -0.3210,
         0.1912, -0.0605, -0.2135,  0.1815,  0.2225, -0.0769,  0.0436,  0.0426,
         0.0520, -0.1147,  0.1500, -0.1225, -0.0742,  0.1197,  0.1429,  0.1736,
         0.1802, -0.2868,  0.1878,  0.2693, -0.0378, -0.0430, -0.1295,  0.0281,
        -0.0408,  0.1477, -0.0142,  0.0383,  0.0363,  0.3557,  0.2661,  0.0890,
        -0.1227, -0.0353, -0.0621, -0.0224,  0.1996, -0.1658,  0.1955, -0.2141,
         0.1688,  0.1162, -0.0088,  0.21

In [129]:
similar_images = annoy_index.get_nns_by_vector(query_img_embedding, 20, include_distances=True)
similar_image_labels = [annoy_index_to_label[str(i)] for i in similar_images[0]]
similar_image_labels

['./dataset/train/salad/5871.png',
 './dataset/train/salad/6267.png',
 './dataset/train/salad/1833.png',
 './dataset/train/salad/476.png',
 './dataset/train/salad/1010.png',
 './dataset/train/salad/344.png',
 './dataset/train/salad/3825.png',
 './dataset/train/salad/2577.png',
 './dataset/train/salad/1202.png',
 './dataset/train/salad/1815.png',
 './dataset/train/salad/3669.png',
 './dataset/train/salad/6315.png',
 './dataset/train/salad/110.png',
 './dataset/train/salad/1280.png',
 './dataset/train/salad/6849.png',
 './dataset/train/salad/122.png',
 './dataset/train/salad/4671.png',
 './dataset/train/salad/5145.png',
 './dataset/train/salad/2469.png',
 './dataset/train/salad/6747.png']

In [80]:
similar_images

([106,
  609,
  668,
  187,
  675,
  637,
  3,
  638,
  24,
  124,
  571,
  99,
  70,
  118,
  13,
  643,
  113,
  610,
  895,
  611],
 [1.0725980997085571,
  1.076981544494629,
  1.086986780166626,
  1.1012552976608276,
  1.1056897640228271,
  1.114335298538208,
  1.12714684009552,
  1.134081482887268,
  1.137354850769043,
  1.1411970853805542,
  1.1431547403335571,
  1.1471046209335327,
  1.1527646780014038,
  1.1545040607452393,
  1.159005045890808,
  1.1607229709625244,
  1.1627691984176636,
  1.1657835245132446,
  1.1728541851043701,
  1.1770964860916138])